In [8]:
import networkx as nx
import urllib.request
import json
import pandas as pd
import re
import pickle
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

### Now that we cleaned up the data we can build the network.

In [17]:
# Load data from data cleaner
edges = pd.read_pickle('LargeData/edges.pkl')
submissions = pd.read_pickle('LargeData/submissions.pkl')


DirectedGraph = nx.MultiDiGraph()

# Create nodes for each author and add all submissions to it
for post in tqdm(submissions):
    author_name = post["author"]
    author_flair = post["flair"]
    submission = post["text"]
    submissions_id = post["id"]

    if author_name not in DirectedGraph.nodes():

        DirectedGraph.add_node(author_name,flair = author_flair,submissions = [submission],submissions_id = [submissions_id])
    else:
        DirectedGraph.nodes[author_name]['submissions'].append(submission)
        DirectedGraph.nodes[author_name]['submissions_id'].append(submissions_id)

# Create a node for authors that only have comments
for post in tqdm(edges):
    author_name = post["author"]
    author_flair = post["flair"]

    # Add author to graph if not already in it
    if author_name not in DirectedGraph.nodes:
        DirectedGraph.add_node(author_name,flair=author_flair,submissions=[],submissions_id=[])

# Create edges between authors 
for post in tqdm(edges):
    author_name = post["author"]
    author_flair = post["flair"]
    addressee = post["addressee"]
    comment = post["comment"]
    comment_id = post["id"]

    if addressee not in DirectedGraph.nodes:
        #Don't add the edge if the addressee is not in the graph because they were deleted
        #print("No addressee  at comment: ",comment_id, " to : ",addressee)
        continue
    else:
        DirectedGraph.add_edge(author_name,addressee,comment=comment,comment_id=comment_id)
    


100%|██████████| 593986/593986 [00:07<00:00, 77897.10it/s]


### Save data

In [18]:
# Save authors as a pickle file
pickle.dump(DirectedGraph, open('LargeData/Graph.pkl','wb'))